In [51]:
#Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
%matplotlib inline

from census import Census
import gmaps
from sqlalchemy import create_engine

In [3]:
api_key= "cb7ed039dfc303ceb774f3adc58bfa50e9eb17db"
c = Census(api_key)


In [133]:
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs.get(("NAME", "B01003_001E","B23025_004E","B19301_001E"), {'for': 'State:*'})


# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B23025_004E": "Employment Count",
                                      "B19301_001E": "Income per Capita",
                                      "NAME": "Name"})
state= census_pd["Name"].str.split(',',1).str[1]
census_pd["State"]= state

# Add in Employment Rate (Employment Count / Population)
census_pd["Employment Rate"] = 100 * \
    census_pd["Employment Count"].astype(
        int) / census_pd["Population"].astype(int)


# # Visualize
# census_pd["Abb"] = ""

# Final DataFrame
census_pd = census_pd[["Name", "Population", "Employment Count", "Employment Rate", "Income per Capita"]]


census_pd.head()

,Name,Population,Employment Count,Employment Rate,Income per Capita
0,Puerto Rico,3468963.0,1040136.0,29.984062,12081.0
1,Alabama,4850771.0,2055509.0,42.374893,25746.0
2,Alaska,738565.0,354045.0,47.936878,35065.0
3,Arizona,6809946.0,2953891.0,43.376130,27964.0
4,Arkansas,2977944.0,1276536.0,42.866353,24426.0


In [134]:
census_pd = census_pd.rename(columns={"Name":"States"})
census_pd.head()


,States,Population,Employment Count,Employment Rate,Income per Capita
0,Puerto Rico,3468963.0,1040136.0,29.984062,12081.0
1,Alabama,4850771.0,2055509.0,42.374893,25746.0
2,Alaska,738565.0,354045.0,47.936878,35065.0
3,Arizona,6809946.0,2953891.0,43.376130,27964.0
4,Arkansas,2977944.0,1276536.0,42.866353,24426.0


In [135]:
census_pd.count()

States               52
Population           52
Employment Count     52
Employment Rate      52
Income per Capita    52
dtype: int64

In [136]:
# Drop all rows with missing information
census_pd = census_pd.dropna(how='any')
census_pd.head()

,States,Population,Employment Count,Employment Rate,Income per Capita
0,Puerto Rico,3468963.0,1040136.0,29.984062,12081.0
1,Alabama,4850771.0,2055509.0,42.374893,25746.0
2,Alaska,738565.0,354045.0,47.936878,35065.0
3,Arizona,6809946.0,2953891.0,43.376130,27964.0
4,Arkansas,2977944.0,1276536.0,42.866353,24426.0


In [137]:
census_pd = census_pd.rename(columns={"States": "states","Population": "population","Employment Count": "emp_count",
                                                         "Employment Rate": "emp_rate", "Income per Capita": "income_per_capita"})
census_pd.head()

,states,population,emp_count,emp_rate,income_per_capita
0,Puerto Rico,3468963.0,1040136.0,29.984062,12081.0
1,Alabama,4850771.0,2055509.0,42.374893,25746.0
2,Alaska,738565.0,354045.0,47.936878,35065.0
3,Arizona,6809946.0,2953891.0,43.376130,27964.0
4,Arkansas,2977944.0,1276536.0,42.866353,24426.0


In [138]:
census_pd = census_pd.drop(census_pd.index[0])
census_pd.head()

,states,population,emp_count,emp_rate,income_per_capita
1,Alabama,4850771.0,2055509.0,42.374893,25746.0
2,Alaska,738565.0,354045.0,47.936878,35065.0
3,Arizona,6809946.0,2953891.0,43.376130,27964.0
4,Arkansas,2977944.0,1276536.0,42.866353,24426.0
5,California,38982847.0,17993915.0,46.158545,33128.0


In [139]:
census_pd.set_index("states", inplace=True)
census_pd.head()

,population,emp_count,emp_rate,income_per_capita
states,,,,
Alabama,4850771.0,2055509.0,42.374893,25746.0
Alaska,738565.0,354045.0,47.936878,35065.0
Arizona,6809946.0,2953891.0,43.376130,27964.0
Arkansas,2977944.0,1276536.0,42.866353,24426.0
California,38982847.0,17993915.0,46.158545,33128.0


In [140]:
census_pd.count()

population           51
emp_count            51
emp_rate             51
income_per_capita    51
dtype: int64

In [143]:
# Store filepath in a variable
file_one = "states.csv"
states_df = pd.read_csv(file_one, encoding="ISO-8859-1")
states_df.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [162]:
states_df = states_df.rename(columns={"State": "state","Abbreviation": "abbreviation",})
states_df.set_index("state", inplace=True)
states_df.head()

,abbreviation
state,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [164]:
file_two = "housing_rate.csv"
housing_rate = pd.read_csv(file_two, encoding="ISO-8859-1")
housing_rate = housing_rate.rename(columns={"State": "state", "2010":"year_2010"})
housing_rate.head()

,state,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,year_2010
0,AK,658430,694245,727550,773297,837885,933706,1013084,1049656,1037857,1044659,517227
1,AL,453752,469052,486102,511127,537481,585134,635231,650963,636567,623674,303442
2,AR,371825,391026,412873,436883,465464,504766,537335,554345,548462,545743,274114
3,AZ,889959,897758,889992,859030,913449,1110553,1241314,1180787,944895,742429,362716
4,CA,1177711,1305707,1441329,1609011,1926909,2255625,2387689,2208476,1733803,1485979,769663


In [165]:
housing_rate = housing_rate.drop(housing_rate.columns[[ 1,2,3,4,5,6,7,8,9,10]], axis =1)
housing_rate.set_index("state", inplace=True)
housing_rate.head()


,year_2010
state,
AK,517227
AL,303442
AR,274114
AZ,362716
CA,769663


In [166]:
connection_string = "postgres:database@localhost:5432/ETL"
engine = create_engine(f'postgresql://{connection_string}')

In [167]:
# Confirm tables
engine.table_names()

['states', 'income', 'housing_rate']

In [168]:
census_pd.to_sql(name='income', con=engine, if_exists='append', index=True)

In [169]:
housing_rate.to_sql(name='housing_rate', con=engine, if_exists='append', index=True)

In [170]:
states_df.to_sql(name='states', con=engine, if_exists='append', index=True)